In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'd:\\Cdac_ML\\Assignments\\Assignment_4'

In [8]:
#entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    estimator: any
    n_estimators: int      
    learning_rate: float     
    random_state: int
    params : any
    target_column: str

In [9]:
from src.Assignment_4.constants import *
from src.Assignment_4.utils.common import read_yaml, create_directories

from sklearn.tree import DecisionTreeClassifier

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_yaml_path,
        params_filepath = params_yaml_path,
        schema_filepath = schema_yaml_path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.AdaBoostClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        # Build estimator here
        estimator1 = DecisionTreeClassifier(
            max_depth=params.estimator.max_depth,
            random_state=params.estimator.random_state
        )

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            estimator=estimator1,
            n_estimators=params.n_estimators,
            learning_rate=params.learning_rate,
            random_state=params.random_state,
            target_column=schema.name,
            params=self.params
        )

        return model_trainer_config


In [10]:
import pandas as pd
import os
from src.Assignment_4 import logger
from sklearn.ensemble import AdaBoostClassifier
import joblib


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)

        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        params = self.config.params.AdaBoostClassifier

        estimator1 = DecisionTreeClassifier(
        max_depth=params.estimator.max_depth,
        random_state=params.estimator.random_state
    )

        rf = AdaBoostClassifier(
            estimator= estimator1,
            n_estimators= params.n_estimators,       
            learning_rate= params.learning_rate,     
            random_state = params.random_state
        )

        rf.fit(train_x, train_y)

        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))


In [11]:
# pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_tr_config = ModelTrainer(config=model_trainer_config)
    model_tr_config.train()
except Exception as e:
    raise e

[2025-11-09 12:34:53,402: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_4\config\config.yaml loaded successfully]
[2025-11-09 12:34:53,405: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_4\params.yaml loaded successfully]
[2025-11-09 12:34:53,409: INFO: common: yaml file: D:\Cdac_ML\Assignments\Assignment_4\schema.yaml loaded successfully]
[2025-11-09 12:34:53,412: INFO: common: created directory at: artifacts]
[2025-11-09 12:34:53,413: INFO: common: created directory at: artifacts/model_trainer]
